In [9]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel("./buscas.xlsx")

# display(tabela_produtos)

In [15]:
nav.get("https://www.google.com/")

produto = "iphone 12 64gb"

# pesquisando pelo produto no google
nav.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(produto, Keys.ENTER)

# entrar na aba shopping
elemento = nav.find_element("xpath", '/html/body/div[4]/div/div[5]/div/div[1]/div/div/div[1]/div/div/div/div/div[1]/div[2]/a')
elemento.click()

# pegando as informações dos produtos
lista_resultados = nav.find_elements("class name", "i0X6df")

for resultado in lista_resultados:
    preco = resultado.find_element("class name", "a8Pemb").text
    nome = resultado.find_element("class name", "tAxDx").text

    elemento_referencia = resultado.find_element('class name', 'bONr3b')
    elemento_pai = elemento_referencia.find_element('xpath', '..') #pegando o elemento que vem logo antes dele ('pai')
    link = elemento_pai.get_attribute('href')
    
    print(preco, nome, link)



  https://www.google.com/url?url=https://www.magazineluiza.com.br/usado-iphone-12-64gb-azul-bom-trocafone-apple/p/djccka1jka/te/ip12/%3Fseller_id%3Dtrocafone%26srsltid%3DAfmBOorv9w7boe8ROg1jZ3NewSi5QAaVrkz0rtVBY8sP5yr7i75K3_mHP0A&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjJ65CTy7yFAxVOqpUCHV_rDVcQguUECIEM&usg=AOvVaw2TpMI_lBaIETIbJdlE7fC0
R$ 2.272,39 Vitrine Iphone 12 Preto 64Gb https://www.google.com/url?url=https://www.carrefour.com.br/vitrine--iphone--12--preto-64gb-mp931849769/p&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjJ65CTy7yFAxVOqpUCHV_rDVcQgOUECJ0M&usg=AOvVaw14pp22W55slVj1N-5L-sjV
R$ 2.272,39 Vitrine. Iphone 12 Branco 64Gb https://www.google.com/url?url=https://www.carrefour.com.br/vitrine-iphone--12--branco--64gb-mp931605276/p&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjJ65CTy7yFAxVOqpUCHV_rDVcQgOUECLAM&usg=AOvVaw3nOC-V_q0eJASeeMh_ecYJ
R$ 2.231,10 iPhone 12 Mini 64 GB - Branco Apple https://www.google.com/url?url=https://www.magazineluiza.com.br/usado-iphone-12-mini-6